**Connect to drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

**PART 1: SET UP THE FOLDERS** 

In [ ]:
import os
import random
import re
from PIL import Image

DATA_PATH = '/content/drive/My Drive/ThesisDL/IOUsemSeg'
MASK_PATH='/content/drive/My Drive/ThesisDL/IOUsemSeg/Nuc_Masks'
FRAME_PATH = '/content/drive/My Drive/ThesisDL/IOUsemSeg/Regression_Composites'



# Create folders to hold images and masks

folders = ['train_frames', 'train_masks', 'val_frames', 'val_masks', 'test_frames', 'test_masks']


for folder in folders:
  os.makedirs(DATA_PATH + '/' + folder)
  
  
# Get all frames and masks, sort them, shuffle them to generate data sets.






In [ ]:
all_frames = os.listdir(FRAME_PATH)
all_masks = os.listdir(MASK_PATH)

In [ ]:
all_frames.sort(key=lambda var:[int(x) if x.isdigit() else x 
                                for x in re.findall(r'[^0-9]|[0-9]+', var)])
all_masks.sort(key=lambda var:[int(x) if x.isdigit() else x 
                               for x in re.findall(r'[^0-9]|[0-9]+', var)])
index=range(len(all_frames))
index2=[i for i in index]

random.seed(230)
random.shuffle(index2)



train_split = int(0.7*len(index2))
val_split = int(0.9 * len(index2))

train_idx = index2[:train_split]
val_idx = index2[train_split:val_split]
test_idx = index2[val_split:]

train_frames=[]
train_masks=[]
val_frames=[]
val_masks=[]
test_frames=[]
test_masks=[]

for idx in train_idx:
  #print(idx)
  train_frames.append(all_frames[idx])
  train_masks.append(all_masks[idx])

for idx in val_idx:
  #print(idx)
  val_frames.append(all_frames[idx])
  val_masks.append(all_masks[idx])

for idx in test_idx:
  #print(idx)
  test_frames.append(all_frames[idx])
  test_masks.append(all_masks[idx])

In [ ]:
random.seed(230)
random.shuffle(all_frames)
random.seed(230)
random.shuffle(all_masks)


# Generate train, val, and test sets for frames

train_split = int(0.7*len(all_frames))
val_split = int(0.9 * len(all_frames))

train_frames = all_frames[:train_split]
val_frames = all_frames[train_split:val_split]
test_frames = all_frames[val_split:]


In [ ]:
def add_frames(dir_name, image):
  
  img = Image.open(FRAME_PATH + '/' +image)
  img.save(DATA_PATH+'/{}'.format(dir_name)+'/'+image)
  
  
  
def add_masks(dir_name, image):
  
  img = Image.open(MASK_PATH+ '/' +image)
  img.save(DATA_PATH+'/{}'.format(dir_name)+'/'+image)


  
  
frame_folders = [(train_frames, 'train_frames'), (val_frames, 'val_frames'), 
                 (test_frames, 'test_frames')]

mask_folders = [(train_masks, 'train_masks'), (val_masks, 'val_masks'), 
                (test_masks, 'test_masks')]



In [ ]:
# Add frames

for folder in frame_folders:
  
  array = folder[0]
  name = [folder[1]] * len(array)
  print(name)
  list(map(add_frames, name, array))
         


In [ ]:
    
# Add masks

for folder in mask_folders:
  
  array = folder[0]
  name = [folder[1]] * len(array)
  
  list(map(add_masks, name, array))

**PART 2: CUSTOM GENERATOR AND NETWORK**

In [ ]:
import cv2
import os
import os
import random
import re
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
tf.compat.v1.enable_eager_execution()

def data_gen(img_folder, mask_folder, batch_size,imsize=128):
  c = 0
  #imsize=512;
  n = os.listdir(img_folder) #List of training images
  m = os.listdir(mask_folder) #List of Mask images
  n,m=shuffle(n,m)
  
  while (True):
    img = np.zeros((batch_size, imsize, imsize, 3)).astype('float')
    mask = np.zeros((batch_size, imsize, imsize, 3)).astype('int')

    for i in range(c, c+batch_size): #initially from 0 to 16, c = 0. 

      train_img = cv2.imread(img_folder+'/'+n[i])/255.
      train_img =  cv2.resize(train_img, (imsize, imsize))# Read an image from folder and resize
      
      

      
      train_mask = cv2.imread(mask_folder+'/'+m[i], cv2.IMREAD_GRAYSCALE)
      train_mask = cv2.resize(train_mask, (imsize, imsize))
      

      #add augmentation
      toFlip=random.random()

      if toFlip<=0.25:
        flipCode=0
      elif toFlip<=0.5 and toFlip>0.25:
        flipCode=-1
      elif toFlip>0.5 and toFlip<=75:
        flipCode=-1
      else:
        flipCode=2
      
      if flipCode!=2:
        train_img = cv2.flip(train_img, flipCode)
        train_mask = cv2.flip(train_mask, flipCode)



      #ADD ONE-HOT ENCODING,
      train_mask=tf.one_hot(train_mask, 3)
      #add ims
      mask[i-c] = train_mask
      img[i-c] = train_img #add to array - img[0], img[1], and so on
    c+=batch_size
    if(c+batch_size>=len(os.listdir(img_folder))):
      c=0
      n,m=shuffle(n,m)
      #print "randomizing again"
    yield img, mask




train_frame_path = DATA_PATH+'/train_frames/'
train_mask_path = DATA_PATH+'/train_masks/'

val_frame_path = DATA_PATH+'/val_frames/'
val_mask_path = DATA_PATH+'val_masks/'


In [ ]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


def unet(pretrained_weights = None,input_size = (128,128,3)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(3, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(3, 1, activation = 'softmax',padding='same')(conv9)

    model = Model(input = inputs, output = conv10)

    #model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = [tf.keras.metrics.MeanIoU(num_classes=3)])
    model.compile('Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model




**PART 3: TRAINING**

In [ ]:
#DEFINE LOSS

from keras import backend as K
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from matplotlib import pyplot as plt
import os
import random
import re



NO_OF_TRAINING_IMAGES = len(os.listdir(train_frame_path))
NO_OF_VAL_IMAGES = len(os.listdir(val_frame_path))

NO_OF_EPOCHS = 500
BATCH_SIZE = 4
imsize=256

weights_path = DATA_PATH+'/test'
#create network
m =unet(pretrained_weights = None,input_size = (imsize,imsize,3))

#load weights
#m =unet(pretrained_weights = weights_path,input_size = (imsize,imsize,3))

opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)


myMetric=tf.keras.metrics.CategoricalAccuracy(name="categorical_accuracy", dtype=None)

#set weights for loss
weights = np.array([1,1,5]) 

loss = weighted_categorical_crossentropy(weights)
m.compile(loss=loss,
              optimizer=opt,
              metrics=[myMetric])

#callbacks: monitor and save
checkpoint = ModelCheckpoint(weights_path, monitor="val_categorical_accuracy", 
                             verbose=1, save_best_only=True, mode='max')

csv_logger = CSVLogger('./log.out', append=True, separator=';')

earlystopping = EarlyStopping(monitor ="val_categorical_accuracy", verbose = 1, patience = 25, mode = 'max')


callbacks_list = [checkpoint, csv_logger, earlystopping]


train_gen = data_gen(train_frame_path,train_mask_path, batch_size = 4,imsize=imsize)
val_gen = data_gen(val_frame_path,val_mask_path, batch_size = 4,imsize=imsize)

results = m.fit_generator(train_gen, epochs=NO_OF_EPOCHS, 
                          steps_per_epoch = (NO_OF_TRAINING_IMAGES//BATCH_SIZE),
                          validation_data=val_gen, 
                          validation_steps=(NO_OF_VAL_IMAGES//BATCH_SIZE), 
                          callbacks=callbacks_list)



**PART 4: Evaluation**

In [ ]:

# list all data in history
history=results
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#define Intersection over Union Metric

def getIOU(trumask,predmask,smooth=0):
  intersection=K.cast(trumask*predmask,dtype='float32')
  union=trumask+predmask
  union=K.cast(union>0,dtype='float32')
  

  I=K.sum(intersection)
  U=K.sum(union)
  
  
  
  if smooth==1:
    IOUS=(I+1)/(U+1);
    return IOUS
  else:
    IOU=I/U
    return IOU

  #print(IOUS)


In [ ]:
#Visualise results and measure IOU for channel 3
test_gen = data_gen(DATA_PATH+'/test_frames/',DATA_PATH+'/test_masks/', batch_size = 4,imsize=imsize)
from google.colab.patches import cv2_imshow
nucIOU=[]
area_real=[]
area_pred=[]
NO_OF_test_IMAGES = len(os.listdir(DATA_PATH+'/test_frames/'))
steps=(NO_OF_test_IMAGES//4)
hold_all_ims=np.zeros((NO_OF_test_IMAGES, imsize, imsize, 3)).astype('float')
hold_all_tru=np.zeros((NO_OF_test_IMAGES, imsize, imsize, 3)).astype('float')
hold_all_pred=np.zeros((NO_OF_test_IMAGES, imsize, imsize)).astype('float')
count=0;
for step in range(steps):
  x,y=next(test_gen)
  ans=m.predict(x,steps=(1))

  for ind in range(4):

    im=x[ind,:,:,:]
    im1=y[ind,:,:,:]
    im2=ans[ind,:,:,:]
    im2=np.argmax(im2,axis=2)
  

    hold_all_ims[count]=im;
    hold_all_tru[count]=im1;
    hold_all_pred[count]=im2

    cv2_imshow(im*125)
    cv2_imshow(im1*250)
    cv2_imshow(im2*125)

    trumask=im1[:,:,2];
    predmask=im2>1;

    nucIOU.append(getIOU(trumask,predmask,1))
    area_real.append(np.sum(trumask))
    area_pred.append(np.sum(predmask))
    count+=1



In [ ]:
#Shew worst/median/best result
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

#calculate locations but set manually for consistency
min_loc=(nucIOU.index(min(nucIOU)))
print(min_loc)
#min_loc=21
max_loc=(nucIOU.index(max(nucIOU)))
print(max_loc)
#max_loc=45
med_loc=(nucIOU.index(find_nearest(nucIOU,np.median(nucIOU))))
print(med_loc)
#med_loc=7

min_tru=hold_all_tru[min_loc]*250
max_tru=hold_all_tru[max_loc]*250
med_tru=hold_all_tru[med_loc]*250

min_pred=hold_all_pred[min_loc]
nu_min_pred=np.zeros((imsize,imsize,3)).astype('float')
nu_min_pred[:,:,0]=min_pred
nu_min_pred[:,:,1]=min_pred
nu_min_pred[:,:,2]=min_pred
nu_min_pred*=125
max_pred=hold_all_pred[max_loc]
nu_max_pred=np.zeros((imsize,imsize,3)).astype('float')
nu_max_pred[:,:,0]=max_pred
nu_max_pred[:,:,1]=max_pred
nu_max_pred[:,:,2]=max_pred
nu_max_pred*=125
med_pred=hold_all_pred[med_loc]
nu_med_pred=np.zeros((imsize,imsize,3)).astype('float')
nu_med_pred[:,:,0]=med_pred
nu_med_pred[:,:,1]=med_pred
nu_med_pred[:,:,2]=med_pred
nu_med_pred*=125


min_im=hold_all_ims[min_loc]*125
max_im=hold_all_ims[max_loc]*125
med_im=hold_all_ims[med_loc]*125

out_file=(cv2.vconcat([cv2.hconcat([min_im,min_tru,nu_min_pred]),cv2.hconcat([med_im,med_tru,nu_med_pred]),cv2.hconcat([max_im,max_tru,nu_max_pred])]))
cv2_imshow(out_file)
fname=(DATA_PATH+'/my_model_' + str(imsize) + '_WIOU_'+ str(np.median(nucIOU))+'.best-worst.png')
print(fname)
cv2.imwrite(out_file,fname)


In [ ]:
##use this unet for imsize 512
#

import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


def unet512(pretrained_weights = None,input_size = (512,512,3)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)

    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)

    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)

    pool21 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv31 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool21)
    conv31 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv31)

    pool3 = MaxPooling2D(pool_size=(2, 2))(conv31)
    conv4 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(2048, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(2048, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)
    up6 = Conv2D(1024, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))

    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    up7 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))

    merge71 = concatenate([conv31,up7], axis = 3)
    conv71 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge71)
    conv71 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv71)
    up81 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv71))

    merge7 = concatenate([conv3,up81], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))

    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))

    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(3, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(3, 1, activation = 'softmax',padding='same')(conv9)

    model = Model(input = inputs, output = conv10)


    model.compile('Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model


In [ ]:
#custom IOU metric for monitoring, optional
def my_metric_fn(x, y):
    IOU=[]
    weights=[1.0,1.0,10.0]
    sumweights=tf.keras.backend.sum(weights)
    for ind in range(4):

   
      im1=x[ind,:,:,:]
      im2=y[ind,:,:,:]
      im2=K.argmax(im2,axis=2)
      channelIOU=[]
      for channelin in range(3):
          trumask=im1[:,:,channelin];
          predmask=im2==channelin;
          unWiegtedIou=getIOU(tf.keras.backend.cast(trumask,dtype='float32'),tf.keras.backend.cast(predmask,dtype='float32'),1)
          channelIOU.append(unWiegtedIou/sumweights*weights[channelin])
      #print(channelIOU)
      IOU.append(tf.keras.backend.mean(tf.keras.backend.cast(channelIOU,dtype='float32')))
      
    return tf.keras.backend.mean(tf.keras.backend.cast(IOU,dtype='float32'))  # Note the `axis=-1`